In [7]:
# import required libraries
import sys

import re
import os
import spacy
import numpy as np
from geopy import Nominatim
nom = Nominatim(user_agent="something") # geopy needs an user_agent argument to work

In [8]:
# set up rd2wgs function
def rd2wgs (x,y):
    """Calculate WGS84 coordinates"""
    x = int(x)
    y = int(y)

    dX = (x - 155000) * pow(10, - 5)
    dY = (y - 463000) * pow(10, - 5)

    SomN = (3235.65389 * dY) + (- 32.58297 * pow(dX, 2)) + (- 0.2475 * pow(dY, 2)) + (- 0.84978 * pow(dX, 2) * dY) + (- 0.0655 * pow(dY, 3)) + (- 0.01709 * pow(dX, 2) *pow(dY, 2)) + (- 0.00738 * dX) + (0.0053 * pow(dX, 4)) + (- 0.00039 * pow(dX, 2) *pow(dY, 3)) + (0.00033 * pow(dX, 4) * dY) + (- 0.00012 * dX * dY)

    SomE = (5260.52916 * dX) + (105.94684 * dX * dY) + (2.45656 * dX * pow(dY, 2)) + (- 0.81885 * pow(dX, 3)) + (0.05594 * dX * pow(dY, 3)) + (- 0.05607 * pow(dX, 3) * dY) + (0.01199 * dY) + (- 0.00256 * pow(dX, 3) *pow(dY, 2)) + (0.00128 * dX * pow(dY, 4)) + (0.00022 * pow(dY,2)) + (- 0.00022 * pow(dX, 2)) + (0.00026 * pow(dX, 5))

    lat = 52.15517 + (SomN / 3600)
    lon = 5.387206 + (SomE / 3600)

    return lat,lon

In [9]:
# set up location list
docLocations = []
docCoordinates = []

In [10]:
# Set up placename function
def placenameFinder(title, text):
    locations = []
    NER = spacy.load("nl_core_news_sm")
    annotated_text = NER(text)
    annotated_title = NER(title)
    estimatedDocCoordinates = []


    for word in annotated_title.ents:
        if word.label_ == "GPE": # GPE = named location
            locations.append(word.text)
    if len(locations) == 0:
        #print("no locations in title")
        for word in annotated_text[0:100].ents:
            if word.label_ == "GPE": # GPE = named location
                locations.append(word.text)
    if len(locations) == 0:
        #print("no locations in first 100 words")
        for word in annotated_text.ents:
            if word.label_ == "GPE": # GPE = named location
                locations.append(word.text)

    for loc in locations:
        print(loc)
        location = nom.geocode(loc)
        estimatedDocCoordinates.append([location.latitude, location.longitude])
    print(estimatedDocCoordinates)


    # Filtering out locations
    for coordinate in estimatedDocCoordinates:
        if not 50.030000 < coordinate[0] < 54.491000 or not -5.790000 < coordinate[1] < 12.700000:
            while estimatedDocCoordinates.count(coordinate):
                estimatedDocCoordinates.remove(coordinate)
    #print(len(estimatedDocCoordinates))

    estimatedCoordinatesArray = np.array(estimatedDocCoordinates)
    
    lat_mean, long_mean = np.mean(estimatedCoordinatesArray,axis=0)
    lat_std, long_std = np.std(estimatedCoordinatesArray, axis=0)

    for coordinate in estimatedDocCoordinates:
        if not lat_mean -2*lat_std < coordinate[0] < lat_mean + 2*lat_std or not long_mean -2*long_std < coordinate[1] < long_mean + 2*long_std:
            while estimatedDocCoordinates.count(coordinate):
                estimatedDocCoordinates.remove(coordinate)
    #print(len(estimatedDocCoordinates))

    # Averaging coordinates of documents with multiple coordinates
    if len(estimatedDocCoordinates) > 1:
        Xsum = 0
        Ysum = 0
        for c in estimatedDocCoordinates:
            Xsum += c[0]
            Ysum += c[1]
        estimatedDocCoordinates = [Xsum/len(estimatedDocCoordinates), Ysum/len(estimatedDocCoordinates)]
        print(estimatedDocCoordinates)

    return estimatedDocCoordinates
    """

    for name in placenameList:
        if name in title:
            location = nom.geocode(name)
            docCoordinates.append([location.latitude, location.longitude])
        else:
            if name in text:
                location = nom.geocode(name)
                docCoordinates.append([location.latitude, location.longitude])
    """

In [11]:
docs = []
folder = r"C:\Users\ewout\Documents\University\Student-Assistantship\DANS docs"
file_location = os.listdir(folder)
for file in file_location:
    docs.append(folder + "\\" + file)

In [12]:
# Test
"""
for doc in docs:
    if "D29601_5" in doc:
        print(doc)
        with open(doc, "r", encoding="utf-8") as f:
            text = f.read()
            docWGS84 = placenameFinder(doc, text)
            print("+++++++++++++++++++++++++++++++++++++++")
            print(docWGS84)

            # Averaging coordinates of documents with multiple coordinates
            if len(docWGS84) > 1:
                Xsum = 0
                Ysum = 0
                for c in docWGS84:
                    Xsum += c[0]
                    Ysum += c[1]
                print(Xsum)
                print(Ysum)
                docWGS84 = [Xsum/len(docWGS84), Ysum/len(docWGS84)]
                print(docWGS84)
"""

'\nfor doc in docs:\n    if "D29601_5" in doc:\n        print(doc)\n        with open(doc, "r", encoding="utf-8") as f:\n            text = f.read()\n            docWGS84 = placenameFinder(doc, text)\n            print("+++++++++++++++++++++++++++++++++++++++")\n            print(docWGS84)\n\n            # Averaging coordinates of documents with multiple coordinates\n            if len(docWGS84) > 1:\n                Xsum = 0\n                Ysum = 0\n                for c in docWGS84:\n                    Xsum += c[0]\n                    Ysum += c[1]\n                print(Xsum)\n                print(Ysum)\n                docWGS84 = [Xsum/len(docWGS84), Ysum/len(docWGS84)]\n                print(docWGS84)\n'

In [13]:
# main script
errorcount = 0
for doc in docs:
    try:
        with open(doc, "r", encoding="utf-8") as f:
            docCoordinates = []
            
            # Finding all coordinates in text:
            text = f.read()
            docCoordinates += re.findall(r"[yxXY]\D{,4}\d{,4}[.,]\d{3},?\d?.{,4}[yxXY]\D{,4}\d{,4}[.,]\d{3},?\d?", text)
            docCoordinates += re.findall(r"\d{2,3}[.,]\d{2,3}.?/.?\d{2,3}[.,]\d{2,3}", text)
            docCoordinates += re.findall(r"\d{5,6}.?/.?\d{5,6}", text)

            if len(docCoordinates) == 0:
                docWGS84 = placenameFinder(doc, text)
            else:
                # Converting coordinates to usable format
                for i, coordinate in enumerate(docCoordinates):
                    coordinate = coordinate.replace(",", "").replace(".", "")
                    coordinate = re.findall(r"\d+", coordinate)
                    docCoordinates[i] = coordinate
            
                # Converting Rijksdriehoek coordinates to WGS 84 coordinates
                docWGS84 = rd2wgs(docCoordinates[0][0], docCoordinates[0][1])

            print("document:" + doc)
            print("docWGS84: " + str(docWGS84))

            # Averaging coordinates of documents with multiple coordinates
            if len(docWGS84) > 2:
                Xsum = 0
                Ysum = 0
                for c in docWGS84:
                    Xsum += c[0]
                    Ysum += c[1]
                docWGS84 = (Xsum/len(docWGS84), Ysum/len(docWGS84))

            # Listing langtide and longitude of each document
            docLocations.append("lan: "+ str(docWGS84[0]) + ", long: " + str(docWGS84[1]) + " in " + str(doc))
            docWGS84 = []

    except Exception as error:
        errorcount += 1
        print("an error occurred in "+ doc)
        print("Error #" + str(errorcount) + ": " + str(error))

[]
an error occurred in C:\Users\ewout\Documents\University\Student-Assistantship\DANS docs\D17109_DeSteekproef_20060219_Nooitgedacht_Hof_van_Saksen_boorstaten.txt
Error #1: cannot unpack non-iterable numpy.float64 object
document:C:\Users\ewout\Documents\University\Student-Assistantship\DANS docs\D17110_DeSteekproef_20060615_Klazienaveen_Kloostermanswijk_rapport.txt
docWGS84: (52.719628880569154, 6.9857156028037934)
document:C:\Users\ewout\Documents\University\Student-Assistantship\DANS docs\D17113_DeSteekproef20060103_Drijber_rapport.txt
docWGS84: (52.79141592356087, 6.5463780953008675)
document:C:\Users\ewout\Documents\University\Student-Assistantship\DANS docs\D20650_RAAP-NOTITIE_274.txt
docWGS84: (52.23386199592088, 4.443157261934848)
document:C:\Users\ewout\Documents\University\Student-Assistantship\DANS docs\D21144_RAAP-NOTITIE_821.txt
docWGS84: (52.1093616833638, 6.6448329218133475)
document:C:\Users\ewout\Documents\University\Student-Assistantship\DANS docs\D21165_RAAP-NOTITIE

C:\Users\ewout\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\ewout\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


meters

an error occurred in C:\Users\ewout\Documents\University\Student-Assistantship\DANS docs\D29595_RA1088-N674V_kbl2_raaien_tot2_deel2_.txt
Error #2: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=meters%0A&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
O.L
O.L
O.L
gemeente IJsselstein
Datum
D.H
[[60.9943157, 75.5139561], [60.9943157, 75.5139561], [60.9943157, 75.5139561], [52.0230641, 5.0549647], [52.9187916, -1.466070582785287], [51.16845155, 17.0281642991244]]
[52.470927849999995, 1.7944470586073566]
document:C:\Users\ewout\Documents\University\Student-Assistantship\DANS docs\D29601_5-RAAPRAPPORT_704.txt
docWGS84: [52.470927849999995, 1.7944470586073566]
[]
an error occurred in C:\Users\ewout\Documents\University\Student-Assistantship\DANS docs\D29603_B4_Overzicht_begravingen_koor.txt
Error #3: cannot unpack non-it

In [14]:
# print output
for location in docLocations:
    print(location)

lan: 52.719628880569154, long: 6.9857156028037934 in C:\Users\ewout\Documents\University\Student-Assistantship\DANS docs\D17110_DeSteekproef_20060615_Klazienaveen_Kloostermanswijk_rapport.txt
lan: 52.79141592356087, long: 6.5463780953008675 in C:\Users\ewout\Documents\University\Student-Assistantship\DANS docs\D17113_DeSteekproef20060103_Drijber_rapport.txt
lan: 52.23386199592088, long: 4.443157261934848 in C:\Users\ewout\Documents\University\Student-Assistantship\DANS docs\D20650_RAAP-NOTITIE_274.txt
lan: 52.1093616833638, long: 6.6448329218133475 in C:\Users\ewout\Documents\University\Student-Assistantship\DANS docs\D21144_RAAP-NOTITIE_821.txt
lan: 52.10829525846016, long: 6.164519817778063 in C:\Users\ewout\Documents\University\Student-Assistantship\DANS docs\D21165_RAAP-NOTITIE-081.txt
lan: 52.470927849999995, long: 1.7944470586073566 in C:\Users\ewout\Documents\University\Student-Assistantship\DANS docs\D29601_5-RAAPRAPPORT_704.txt
lan: 52.97971757523593, long: 5.607111863181326 i